In [ ]:
spark

In [ ]:
sc

### Instalação no ambiente do docker desktop
Instalar o netcat para utilizar uma porta no jupyter-spark para poder enviar dados via tcp.

docker exec -it jupyter-spark bash
root@jupyter-spark:/# apt update

root@jupyter-spark:/# apt install netcat

root@jupyter-spark:/# 

#### DStream não termina. Recebe o dado e processa!

### Exercício DStream - WordCount

In [ ]:
# criando um diretório no HDFS

!hdfs dfs -mkdir /user/feliciani/stream

In [ ]:
# importando o Streaming Context

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from time import sleep

In [ ]:
# criando o local e o nome da aplicação
conf = SparkConf().setMaster("local[*]").setAppName("Dstream WordCount")

sc = SparkContext.getOrCreate(conf)

# criando o Streaming Context no spark de 10 em 10 segundos
ssc = StreamingContext(sc,10)

In [ ]:
# criando o Dstream a partir de uma porta de entrada tcp
# localhost, neste caso é o container jupyter-spark
# porta 9989 pode ser utilizado, por que o netcat foi instaldo com sucesso

dstream = ssc.socketTextStream("localhost",9998)


In [ ]:
# pegará a mapeará a linha e fará um split por espaço

word_count = dstream.flatMap(lambda linha: linha.split(" "))\
                    .map(lambda palavra: (palavra,1))\  # cada palavra que foi separada com o split será mapeada em chave e valor
                    .reduceByKey(lambda chave1, chave2: chave1 + chave2) # contando as palavras

In [ ]:
# vai printar os Dstream que estão chegando

word_count.pprint()

#### no terminal container do jupyter-spark digitar
#### root@jupyter-spark:/# nc -lp 9998
(vai associar os dados que estão chegando a porta 9998,  l = listem, p = porta)

Todo dado inserido nesse terminal, será printado no jupyter notebook abaixo do comando ssc.start()

In [ ]:
ssc.start() # inicia o processo e print a cada 10 segundos

sleep(50) # depois de 5 prints vai parar, e ir para o stop()

ssc.stop() # pára o processo